# Preliminaries for Using the Marlowe Starter Kit

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

This notebook provides instructions on setting up signing keys and addresses for this starter kit. It covers the following information:

- Check for cli tools and environment variables
- Creating addresses and signing keys
    - The faucet
    - The lender
    - The borrower
    - The mediator
- Obtaining test ada
- Fund the addresses of the parties
    - Using Daedalus or a web-browser wallet
    - Using a local faucet at the command line
    
[A video works through this Jupyter notebook.](https://youtu.be/hGBmj9ZrYHs)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

## Command Line Tools and Environment

This section introduces the `check-tools-and-env.sh` script, located in the *scripts* folder. This script is designed to validate that the required command line tools and environment settings are correctly set.

You can delve into the [scripts folder notebook](../scripts/scripts.ipynb) to find different helpers that are used accross the different notebooks.

In [ ]:
export SCRIPTS=$(realpath ../scripts)
export KEYS=$(realpath ../keys)
source $SCRIPTS/check-tools-and-env.sh

## Creating Addresses and Signing Keys

The [Cardano Developers Portal](https://developers.cardano.org/docs/stake-pool-course/handbook/keys-addresses/) contains instructions for creating addresses and signing keys.

This starter kit uses the following addresses:
- A local *Faucet* used to fund the other parties.
- The *Lender* party for the examples in this starter kit.
- The *Borrower* party for the examples in this starter kit.
- The *Mediator* party for some examples in this starter kit.

The instructions below detail how to create signing keys and addresses for these parties. 
***IMPORTANT:*** The [keys/](../keys/) folder holds the signing keys that will be created for interacting with the Marlowe contract. If you delete or lose these files, then you also forever lose the test ada stored at those addresses. Either backup these files or, after running the tutorials, send the remaining test ada back to a more permanent wallet or return it to the faucet.

### The Faucet

In this section we create a wallet that will act as a faucet we own. 

In [ ]:
FAUCET_SKEY=$KEYS/faucet.skey
FAUCET_VKEY=$KEYS/faucet.vkey

if [[ ! -e "$FAUCET_SKEY" ]]
then
  echo "Creating Faucet key"
  cardano-cli address key-gen \
    --signing-key-file "$FAUCET_SKEY" \
    --verification-key-file "$FAUCET_VKEY"
else 
  echo "Faucet key already present"
fi

Compute the faucet's address on the testnet.

In [ ]:
FAUCET_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$FAUCET_VKEY" )
echo "$FAUCET_ADDR" > $KEYS/faucet.address
echo "FAUCET_ADDR = $FAUCET_ADDR"

### The Lender

To create the rest of the wallets we use the `create-test-wallet.sh` script, which will setup the signing key, address and also a recovery phrase to use together with a wallet

In [ ]:
source $SCRIPTS/create-test-wallet.sh lender

### The Borrower

Do the same for the borrower

In [ ]:
source $SCRIPTS/create-test-wallet.sh borrower

### The Mediator

Do the same for the mediator

In [ ]:
source $SCRIPTS/create-test-wallet.sh mediator

## Obtaining Test Ada

In order to execute transactions on a Cardano network, one needs the native currency ada to pay fees and use as funds. There are the faucets for the public testnets at https://docs.cardano.org/cardano-testnet/tools/faucet where one can obtain test ada daily.

Optionally, it can be convenient to centrally manage funds with a [web-browser wallets](https://builtoncardano.com/ecosystem/wallets): be sure to select the correct public testnet if using one of these wallets.

If you will be using a local faucet (recommended), then send test ada to the faucet address created in the previous section. Otherwise, send the test ada to the web-browser wallet.

In [ ]:
echo "FAUCET_ADDR = $FAUCET_ADDR"
echo
$SCRIPTS/cardano-scan-address.sh $FAUCET_ADDR

## Fund the Addresses of the Parties

We'll fund each address with 1000 test ada.

In [ ]:
echo "LENDER_ADDR = $LENDER_ADDR"
echo "BORROWER_ADDR = $BORROWER_ADDR"
echo "MEDIATOR_ADDR = $MEDIATOR_ADDR"

### Using Daedalus or a Web-Browser Wallet

If you already have a wallet wallet that contains test ada, then you can just send the funds to the addresses of the keys that we created in the previous section.


### Using a Local Faucet at the Command Line

One can use `cardano-cli` or `marlowe-cli` send funds to an address. Here we use `marlowe-cli`.

***If you have just funded `FAUCET_ADDR` with ada, you may have to wait a couple of minutes before that transaction is confirmed. If the command below fails, then retry it until it succeeds.***

In [ ]:
# Note that `FAUCET_ADDR` must have already been funded with test ada.

# 1 ada = 1,000,000 lovelace
ADA=1000000

# Send 1000 ada
AMOUNT=$((1000 * ADA))

# Execute the transaction.
marlowe-cli util fund-address \
 --lovelace "$AMOUNT" \
 --out-file /dev/null \
 --source-wallet-credentials "$FAUCET_ADDR":"$FAUCET_SKEY" \
 --submit 600 \
 "$LENDER_ADDR" "$BORROWER_ADDR" "$MEDIATOR_ADDR"

Alternatively you can use the `fund-wallets.sh` script as a single liner
> NOTE: The following script does the same as the above, no need to do it twice.

In [ ]:
$SCRIPTS/fund-wallets.sh $LENDER_ADDR $BORROWER_ADDR $MEDIATOR_ADDR

See that the addresses have indeed been funded:

In [ ]:
echo
echo "Lender"
$SCRIPTS/cardano-scan-address.sh $LENDER_ADDR
echo
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"
echo
echo
echo
echo

echo "Borrower"
$SCRIPTS/cardano-scan-address.sh $BORROWER_ADDR
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"
echo
echo
echo
echo

echo "Mediator @ $MEDIATOR_ADDR"
$SCRIPTS/cardano-scan-address.sh $MEDIATOR_ADDR
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"
echo

## Return funds to the faucet

If you are using a local faucet it is recommended to return the test ADA after running the desired notebooks

In [ ]:
$SCRIPTS/return-assets-to-faucet.sh lender
$SCRIPTS/return-assets-to-faucet.sh borrower
$SCRIPTS/return-assets-to-faucet.sh mediator